In [14]:
import numpy as np
import netCDF4 as nc

# NetCDF 파일 열기
#ncfile = nc.Dataset(r"D:\KMK_DATA\NWP\l015_v070_erlo_unis_20kind.2023041000.nc")
#ncfile = nc.Dataset("../../../../../DATA/l015_v070_erlo_unis_20kind.2023040300.nc")
ncfile = nc.Dataset("../../../../../DATA/test.nc")

print(ncfile)

#cdo remapnn,lon=127.5_lat=35.5 l015_v070_erlo_unis_20kind.2023040300.nc test.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 2.2.3 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.0
    History: Mon Apr  3 15:36:26 2023: ncrcat ./l015_v070_erlo_unis_h000.2023040300.nc ./l015_v070_erlo_unis_h001.2023040300.nc ./l015_v070_erlo_unis_h002.2023040300.nc ./l015_v070_erlo_unis_h003.2023040300.nc ./l015_v070_erlo_unis_h004.2023040300.nc ./l015_v070_erlo_unis_h005.2023040300.nc ./l015_v070_erlo_unis_h006.2023040300.nc ./l015_v070_erlo_unis_h007.2023040300.nc ./l015_v070_erlo_unis_h008.2023040300.nc ./l015_v070_erlo_unis_h009.2023040300.nc ./l015_v070_erlo_unis_h010.2023040300.nc ./l015_v070_erlo_unis_h011.2023040300.nc ./l015_v070_erlo_unis_h012.2023040300.nc ./l015_v070_erlo_unis_h013.2023040300.nc ./l015_v070_erlo_unis_h014.2023040300.nc ./l015_v070_erlo_unis_h015.2023040300.nc ./l015_v070_erlo_unis_h016.2023040300.nc ./l015_v070_erlo_unis_h017.2023040300.nc ./l015_v070_erlo_unis_h018.20230

In [8]:
# from nco import Nco
# test = Nco()


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [33]:
import pandas as pd 
lat = nco.ncra(input=r"D:\KMK_DATA\NWP\l015_v070_erlo_unis_20kind.2023041000.nc", returnArray=True).variables["latitude"][:]
#lat = pd.DataFrame(np.array(ncfile.variables["latitude"]))
lon = pd.DataFrame(np.array(ncfile.variables["longitude"]))

AttributeError: module 'nco' has no attribute 'ncra'

In [1]:
!pip install nctoolkit

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Obtaining dependency information for nctoolkit from https://files.pythonhosted.org/packages/37/18/a25d010578ea41e1032f55dce34325e68b2ee534a1c8ed5763bf771679d6/nctoolkit-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for netCDF4 from https://files.pythonhosted.org/packages/80/c9/e0419aa3343453a9f5ecbdd051ceb89f93a804beeb79f7dd7ffd023a6e68/netCDF4-1.6.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for hvplot from https://files.pythonhosted.org/packages/1c/4a/87bf45985ba1fa9614ca9da82e4d8b91e25952da4eb72239ddb8bc4d07f9/hvplot-0.8.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for panel from https://files.pythonhosted.org/packages/a0/b6/305f4ef5aa07f9d672a257362

In [ ]:
# import nctoolkit as nc
# data = nc.open_data(infile)
# data.to_latlon(lon = [lon_min,lon_max],lat=[lat_min,lat_max], res =[0.083, 0.083])